### Introduction: Applying Computational Thinking to Personal Finance and Portfolio Analysis
In an era where financial independence is closely tied to the ability to make strategic investment decisions, understanding the mechanics of saving and investing has become a critical life skill. Yet, while many individuals understand the importance of saving, few have the tools or knowledge required to evaluate how different financial strategies—and especially investment decisions—impact long-term outcomes. This tutorial addresses that gap by introducing students to essential concepts in personal finance and portfolio analysis through a computational and data-driven approach.

This tutorial begins by building a foundation in core savings principles. Learners will explore the functionality of common financial instruments such as high-yield savings accounts, 401(k)s, and Roth IRAs. Through interactive inputs and simulations, students will see how interest rates, compound growth, and tax advantages affect returns over time. These early exercises aim to demonstrate the value of starting early, contributing consistently, and choosing the right savings vehicles for one’s personal financial goals.

Building on this foundation, the tutorial then transitions into a more advanced, investment-focused module: constructing and evaluating a personalized stock portfolio. Students will be guided through the process of:
- Selecting their own breakdown of stocks to form a diversified portfolio based on individual risk preferences or interests.

- Evaluating the strength of their chosen portfolio using historical data and key financial metrics.

- Simulating potential future outcomes through Monte Carlo simulations, which model thousands of possible future scenarios to account for uncertainty and market volatility.

- Optimizing the stock allocation using algorithmic methods that balance return potential with risk, aiming for the most efficient use of investment capital.

- Rechecking the portfolio’s strength after optimization, to reflect the improvements made and demonstrate the impact of computational adjustments on financial performance.


Throughout this tutorial, students will employ principles from computer science—such as data modeling, simulations, algorithmic thinking, and data visualization—to analyze financial data and make informed decisions. By the end of the tutorial, students will have developed not only a deeper understanding of financial literacy but also a practical appreciation for how computational tools can empower individuals to manage their personal wealth more effectively.

This interdisciplinary exploration is designed to equip learners with a strong foundation in both personal finance and applied computation—skills that are increasingly essential in today’s data-rich, financially complex world.

### Disclaimer

**No Guarantee of Accuracy:** While Isabel, Luna, and Nirantheri strive to provide accurate and up-to-date information, they do not guarantee the accuracy, completeness, or reliability of any content. Users should independently verify any information before making financial or investment decisions based on it.


**Investment Risks:** Investing involves inherent risks, including but not limited to market fluctuations, economic uncertainty, geopolitical events, and individual asset performance. Past performance is not indicative of future results, and no content provided implies a guarantee of investment success.

**Limitation of Liability:** Mentions of specific financial products, services, companies, or securities within the content do not constitute endorsements or recommendations. Users are responsible for conducting their own research and due diligence.

**No Liability:** Isabel, Luna, Nirantheri, and their affiliates, partners, or contributors shall not be held liable for any loss, damage, or expense resulting from the use of or reliance on the provided information. All investment decisions made based on this content are the sole responsibility of the user.

**Disclaimer Updates:** This disclaimer is subject to change without notice. Users are responsible for reviewing it periodically to stay informed of any updates.



### using old material : Nirantheri


The code to show the importance of investing vs savings (just draw out each of the ira, 401k, hysa along with no investment to show)
How to use information.md?

### setup

include all packages for setting up (pyfolio, package)

In [2]:
# all libraries

# import pyfolio
import yfinance as yf
import numpy as np
import scipy
import ipywidgets as widgets
import pandas as pd
from scipy.optimize import minimize
from IPython.display import display
import altair as alt
import quantstats.stats as stats

import matplotlib.pyplot as plt
%matplotlib inline

### Choosing stocks for a portfolio: Nirantheri

We've given you a list of stocks to choose from. Ctrl+Click the ones you want in your portfolio. 

In [44]:
codes = {"Apple": 'AAPL',
        "Costco": 'COST',
        "Microsoft": 'MSFT',
        "Google": 'GOOG',
        "Nvidia": 'NVDA',
        "Walmart": 'WMT',
        "Tesla": "TSLA",
        "AMC Entertainment": 'AMC',
        "S&P 500": '^GSPC',
        "Dow Jones Industrial": '^DJI',
        "Nasdaq": '^IXIC',
        "Vanguard S&P 500 ETF": 'VOO',
        "Fidelity 500 Index Fund": 'FXAIX',
        "Gold":'GLD'}

codes.keys()


choices = widgets.SelectMultiple(
    options=codes.keys(),
    value=[],
    rows=20,
    description='Stocks',
    disabled=False
)

display(choices)


SelectMultiple(description='Stocks', options=('Apple', 'Costco', 'Microsoft', 'Google', 'Nvidia', 'Walmart', '…

### Picking your own breakdown of stocks : Nirantheri

Without rerunning the previous code chunk, run the next one and input your allocation amounts as a decimal ex (0.3 or .5). Make sure they add up to 1!
<!-- 
[setup using the above code chunk](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Tabs) -->

In [45]:
# get breakdown

# get codes for further down
portfolio = []

for i in range(len(choices.value)):
    portfolio.append(codes[choices.value[i]])

# pick distribution

tab_contents = ["Value"] * len(choices.value)
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = choices.value

tab


Tab()

The following code is to match the allocation amounts to the stock codes given the inputs from above. 

In [46]:
allocations = {}

for i in range(len(tab.children)):
    allocations[portfolio[i]]=tab.children[i].value


### Interested in testing out your own set of stocks?

Feel free to create your own allocation dictionary. Make sure each entry is of the format "STOCK TICKER": allocation amount. Fill it in as you desire and click run to make sure that your custom allocations are used, even if you didn't run the above lines. Otherwise our default set of stocks will be used!

In [47]:
# Here's an example of what your dictionary can look like

# {'AAPL':'0.4', 'MSFT':'0.3',  '^IXIC':'0.3'}

allocations = {}

### pulling the yf data: Luna

Assume that the list of stocks will come in as a set of stock codes so you can write a loop to get each of them

In [ ]:
#pulled a dictionary of different stocks, keys are tickers? i cant think of a better way to store them because the list is variable so its a bit of an issue

# testList = ['AAPL', 'MSFT', 'GOOG', 'NVDA', '^GSPC', '^DJI', '^IXIC']
if len(allocations.keys())==0: # default settings
    portfolio = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'BIRK']
    allocations={'AAPL':'0.2', 'MSFT':'0.2', 'GOOG':'0.2', 'NVDA':'0.2', 'BIRK':'0.2'}

testList=portfolio

# testString = ' '.join(testList)

# tickersPull = yf.Tickers(testString)

stock_min = [yf.Ticker(x).history(period='max').index.min() for x in testList] # get the earliest date (IPO) of each stock

# either the last 10 years or the last x years based on the stock with the most recent IPO
later_start_date = max(max(stock_min), pd.Timestamp('2015-01-01', tz='America/New_York')) 

# Pull data for each stock starting from the latest start date

dataframes = {}
for x in testList:
    ticker = yf.Ticker(x)
    dataframes[x] = ticker.history(period='1mo', start=later_start_date, auto_adjust=True) # (auto_adjust accounts for splits and dividends)
    # print(dataframes[x])


## Checking the returns with this : Nirantheri

Using the yf data, we will check what returns looked like over time.

#### TODO: should we try to project or work with previous data??

https://blog.mlq.ai/python-for-finance-portfolio-optimization/


## How do we calculate returns?

To calculate the returns, we want to find the adjusted value of each stock amount from the date of the initial investment. We can write a function to do this with a set of inputs-- an initial portfolio value, the set of dataframes for each stock, and the breakdown of allocations  <!-- and perhaps even an initial date of investment.  -->

In [49]:
def calculate_returns(initial_portfolio_val, dataframe, allocations):
    all_pos_vals = []

    for stock_name in dataframe:
        # grab the dataframe for a single stock
        stock_df = dataframe[stock_name]
        # create normed return column
        stock_df['Normed Return'] = stock_df['Close'] /stock_df.iloc[0]['Close']

        # use normed return to adjust the percentage of portfolio held
        allocation = float(allocations[stock_name])  # Convert allocation to float
        stock_df['Allocation'] = stock_df['Normed Return']*allocation

        # find value of stock at each date
        stock_df['Position Value'] = stock_df['Allocation']*initial_portfolio_val

        # add to list of all position values
        all_pos_vals.append(stock_df['Position Value'])


    # concatenate the list of position values
    portfolio_val = pd.concat(all_pos_vals, axis=1)

    # set the column names
    portfolio_val.columns = portfolio

    # add a total portfolio column
    portfolio_val['Total'] = portfolio_val.sum(axis=1)

    # changing date to column not index
    portfolio_val = portfolio_val.reset_index()

    return portfolio_val

Then we can create a graphical representation of each of the stocks' growths over time as well as the portfolio's overall growth. To get an overall number rather than using every single data point, we can use a resampling function to grab the last day of each month in the data.

In [50]:
def graph_values(portfolio_values, graph_portfolio, graph_stocks):
    """graph_portfolio is a bool to show only the total portfolio, graph_stocks is to graph all stocks."""

    # Resample the portfolio_val DataFrame to only include the last date of each month
    portfolio_val = portfolio_values.resample('ME', on='Date').last().reset_index()

    if graph_portfolio:
        portfolio_val['Total'].plot(figsize=(10,8))
    if graph_stocks:
        portfolio_val.drop('Total', axis=1).set_index('Date').plot(figsize=(10,8))


    print("total value", portfolio_val['Total'].iat[-1])


Now that we've created a projection of how our data has been growing over time, let's see what to expect!

In [51]:
returns = calculate_returns(1e6, dataframes, allocations)

graph_values(returns, False, False)

total value 1345721.7594906543


In [43]:
# TODO: Implement the altair version which has more interactivity

# # need to pivot the data
# # print(portfolio_val.head())
# print(portfolio_val.info())
# portfolio_val.drop("^IXIC", axis=1)
# pivoted_data = portfolio_val.melt(id_vars="Date", var_name="Stock", value_name="Value")

# pivoted_data.head()


# alt.Chart(pivoted_data).mark_line().encode(
#     alt.X("Date:T", title="Date"),
#     alt.Y("Value:Q", title="Portfolio Value"),
#     alt.Color("Stock:N", title="Stock")
# )


# plot our portfolio

### check strength of portfolio : Isabel
pyfolio: https://www.pyquantnews.com/the-pyquant-newsletter/create-beautiful-strategy-tear-sheets-pyfolio-reloaded
can also use quantstats



pyfolio: get the tear sheets--> then use pandas to pull specific metrics (annual return, etc) and then have a md cell which breaks down what each of them means

## 🧾 Portfolio Strength Breakdown

**📈 CAGR (Compound Annual Growth Rate):**  
Shows the average annual growth of your portfolio over time. The higher, the better — it reflects long-term performance.

**📊 Sharpe Ratio:**  
Measures your return per unit of risk. A Sharpe ratio above 1.0 is generally considered good. It means you're getting rewarded well for the volatility you’re taking on.

**📉 Max Drawdown:**  
The worst loss your portfolio experienced from peak to bottom. A lower number means your portfolio didn’t crash too hard.

**📈 Volatility:**  
Reflects how much your portfolio's value fluctuates. High volatility can mean high risk — or high opportunity.

**📅 Average Daily Return:**  
The average return your portfolio gained (or lost) per trading day. Helps you see how it behaves short term.

---

These metrics help you assess whether your portfolio is efficient, risky, or well-balanced — and give you data to improve it over time.

We can write a function to return these values for us.


In [17]:
def portfolio_stats(returns):
    # Ensure datetime is clean
    returns.set_index('Date', inplace=True)

    # Daily returns from portfolio value
    daily_returns = returns['Total'].pct_change().dropna()

    # Grab specific stats
    cagr = stats.cagr(daily_returns)
    sharpe = stats.sharpe(daily_returns)
    drawdown = stats.max_drawdown(daily_returns)
    volatility = stats.volatility(daily_returns)
    avg_return = stats.avg_return(daily_returns)

    return {"cagr":cagr, "sharpe":sharpe, "drawdown":drawdown, "volatility":volatility, "avg_return":avg_return}


Now that we've written a function for this, we can analyze the returns of our portfolio from above.

In [18]:
portfolio_info = portfolio_stats(returns)

print(f"📈 CAGR (Annual Return): {portfolio_info["cagr"]:.2%}")
print(f"📊 Sharpe Ratio: {portfolio_info["sharpe"]:.2f}")
print(f"📉 Max Drawdown: {portfolio_info["drawdown"]:.2%}")
print(f"📈 Volatility: {portfolio_info["volatility"]:.2%}")
print(f"📅 Average Daily Return: {portfolio_info["avg_return"]:.2%}")

📈 CAGR (Annual Return): 13.97%
📊 Sharpe Ratio: 0.89
📉 Max Drawdown: -32.66%
📈 Volatility: 24.68%
📅 Average Daily Return: 0.09%


### montecarlo simulations : tbd

uses yahoo finance package and others to create montecarlo simulation for modeling

montecarlo https://medium.com/analytics-vidhya/monte-carlo-simulations-for-predicting-stock-prices-python-a64f53585662

### optimizing the set of stocks : luna

use scipy

optimizing your investment: https://medium.com/@ethan.duong1120/python-powered-portfolio-optimization-achieving-target-returns-through-weight-optimization-fc5163e5c9c6


In [19]:

# testList = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'SBUX', '^GSPC', '^DJI', '^IXIC']

testList = portfolio

# pull the first stock's data; drop all unnecessary columns
ticker = yf.Ticker(testList[0])
df = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
df.drop(columns=['High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], inplace=True)
df.rename({'Open' : 'AAPL'}, inplace=True)

# for every other ticker, add its open value to the existing dataframe
for x in testList[1:]:
    ticker = yf.Ticker(x)
    data = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
    df.insert(len(df.columns), x, data["Open"]) # for whatever reason, this line wont work

#calculate percent returns for each day of each stock
returns_df = df.pct_change(1).dropna()

#operationalize determining portfolio returns
def getPfReturn(weights):
    """
    return is annualized expected return of portfolio
    """
    expRetPortfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    return expRetPortfolio

# start with stocks at equal weights
numStocks = len(returns_df.columns)
initialWeight = [1/numStocks] * numStocks

# return goal?

#TODO: figure out what we should set this value to

targetReturn = .4

# bounds the percentage of each stock we can hold (between 0 and 100%)
bounds = tuple((0,1) for i in range(numStocks))

# ensures the sum of all stock weights is 100% (or 1) in first constraint
# sets goal of minimize function to hit targetReturn
constraints = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
               {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df.mean()) * 250 - targetReturn})

# can we reach our goal with these stocks??
results = minimize(fun=getPfReturn, x0=initialWeight, bounds=bounds, constraints=constraints)

#output
print(results)

optimizedResults = pd.DataFrame(results['x'])

# print our optimized results
getPfReturn(weights=results["x"])
optimizedResults.index = returns_df.columns

optimizedResults.rename(index={optimizedResults.index[0]: portfolio[0]}, inplace=True)

print(optimizedResults)

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 0.2536147351521132
       x: [ 1.000e+00  4.337e-12  4.898e-19]
     nit: 63
     jac: [ 2.536e-01  2.501e-01  1.462e-01]
    nfev: 790
    njev: 59
                  0
AAPL   1.000000e+00
MSFT   4.336723e-12
^IXIC  4.898369e-19


In [22]:
initial_portfolio_val = 1e6

optimized_allocations = optimizedResults[0].to_dict()

optimized_returns = calculate_returns(initial_portfolio_val, dataframes, optimized_allocations)

graph_values(optimized_returns, False, False)

total value 7987933.724932499


*italicized text*### rechecking the strength of your portfolio : isabel's

go back to pyfolio--> will integrate after we recheck the working code

### 📉 Rechecking Strength After Optimization

After optimizing our portfolio allocations to achieve a target return of 40%, we reevaluate the portfolio using updated historical performance metrics.

The results below reflect the **impact of computational adjustments**:

- **CAGR (Annual Return):** Shows the expected annual growth based on optimized allocations.
- **Sharpe Ratio:** Tells us how well we’re balancing risk and reward with the new weights.
- **Max Drawdown:** Indicates if the worst-case dip has improved.
- **Volatility:** Helps assess whether the portfolio became more stable or more erratic.
- **Avg Daily Return:** Confirms whether daily trends look stronger after optimization.

This second performance check shows whether our changes actually led to a more efficient investment strategy.


In [23]:
portfolio_info = portfolio_stats(optimized_returns)

print(f"📈 CAGR (Annual Return): {portfolio_info["cagr"]:.2%}")
print(f"📊 Sharpe Ratio: {portfolio_info["sharpe"]:.2f}")
print(f"📉 Max Drawdown: {portfolio_info["drawdown"]:.2%}")
print(f"📈 Volatility: {portfolio_info["volatility"]:.2%}")
print(f"📅 Average Daily Return: {portfolio_info["avg_return"]:.2%}")

📈 CAGR (Annual Return): 14.97%
📊 Sharpe Ratio: 0.84
📉 Max Drawdown: -38.52%
📈 Volatility: 29.20%
📅 Average Daily Return: 0.10%


After constructing our initial stock portfolio based on user-selected allocations, we evaluated its historical performance using metrics like annual return, Sharpe ratio, and drawdown. This gave us a baseline understanding of how well our portfolio performed under those weights.

We then used optimization techniques (via scipy.minimize) to automatically adjust the stock allocations in order to achieve a target return while minimizing risk. This step simulates what a financial algorithm might do to make our portfolio more efficient.

Finally, we re-evaluated the optimized portfolio using the same performance metrics. By comparing the before-and-after results, we can see whether the optimization actually improved our portfolio — either by increasing returns, reducing risk, or improving the Sharpe ratio (risk-adjusted performance).

This comparison demonstrates the power of computational thinking in making data-driven investment decisions.